## Với bài dùng Langgraph thì nó có 1 hàm có sẵn phục vụ, hỗ trợ cho việc EXTRACT MEMORY. 

Link: [DoanNgocCuong/ai-companion---Long-Term-Memory---Memory-Augmented-AI-Agents: Meet Ava, the WhatsApp Agent](https://github.com/DoanNgocCuong/ai-companion---Long-Term-Memory---Memory-Augmented-AI-Agents)

Sử dụng: LANGGRAPH + SQLite + Qdant để quản lý Short Term và Long Term. 

Để **Memory Extraction Node** xác định và trích xuất các thông tin quan trọng như sở thích, tên, hoặc thông tin cá nhân của người dùng từ nội dung cuộc trò chuyện, nó áp dụng các kỹ thuật xử lý ngôn ngữ tự nhiên (NLP) sau:

1. **Nhận dạng thực thể có tên (Named Entity Recognition - NER):** Kỹ thuật này giúp xác định và phân loại các thực thể trong văn bản thành các nhóm như tên người, địa điểm, tổ chức, v.v. Ví dụ, trong câu "Tôi sống ở Hà Nội và làm việc cho công ty ABC", NER sẽ nhận diện "Hà Nội" là địa điểm và "công ty ABC" là tổ chức.
    
2. **Gán nhãn từ loại (Part-of-Speech Tagging):** Quá trình này gắn nhãn cho từng từ trong câu dựa trên chức năng ngữ pháp của chúng, như danh từ, động từ, tính từ, v.v. Điều này giúp hiểu rõ cấu trúc câu và mối quan hệ giữa các từ.
    
3. **Giải quyết đồng tham chiếu (Coreference Resolution):** Kỹ thuật này xác định khi nào các từ hoặc cụm từ khác nhau đề cập đến cùng một thực thể. Ví dụ, trong hai câu liên tiếp "Anh ấy là một kỹ sư. Anh ấy làm việc tại Google.", "Anh ấy" trong cả hai câu đều đề cập đến cùng một người.
    
4. **Phân tích cú pháp (Parsing):** Phân tích cấu trúc ngữ pháp của câu để hiểu cách các từ được sắp xếp và liên kết với nhau, giúp xác định mối quan hệ giữa các thành phần trong câu.
    

<think>
Tôi sẽ so sánh code trong thư mục studio-integration-version với v3-streamlit-supabase-mem0.py để xác định những điểm khác biệt chính:

1. Kiến trúc ứng dụng:
   - studio-integration-version sử dụng FastAPI để tạo một API endpoint thay vì Streamlit UI
   - v3-streamlit-supabase-mem0.py là ứng dụng web UI sử dụng Streamlit

2. Cách xử lý người dùng:
   - studio-integration-version sử dụng session_id và user_id để phân biệt người dùng
   - v3-streamlit-supabase-mem0.py quản lý người dùng qua UI đăng nhập Streamlit

3. Lưu trữ tin nhắn:
   - studio-integration-version lưu tin nhắn trong bảng "messages" trong Supabase
   - v3-streamlit-supabase-mem0.py chỉ lưu trong session state của Streamlit

4. Sử dụng thư viện:
   - studio-integration-version sử dụng thư viện pydantic_ai, logfire, httpx
   - v3-streamlit-supabase-mem0.py sử dụng trực tiếp OpenAI

5. Xử lý bối cảnh và lịch sử hội thoại:
   - studio-integration-version truy vấn lịch sử hội thoại từ cơ sở dữ liệu
   - v3-streamlit-supabase-mem0.py sử dụng session state của Streamlit

Nếu muốn cải thiện v3-streamlit-supabase-mem0.py để tách riêng người dùng và thêm UI kiểm tra log, tôi sẽ tập trung vào:
1. Lưu trữ tin nhắn trong Supabase thay vì session state
2. Tạo trang admin riêng để xem log và thông tin
3. Cải thiện cách quản lý phiên và phân biệt người dùng
</think>

Sau khi so sánh code trong thư mục `studio-integration-version` với file `v3-streamlit-supabase-mem0.py`, tôi thấy có những khác biệt quan trọng sau:

## Khác biệt chính:

1. **Kiến trúc ứng dụng**:
   - `studio-integration-version` sử dụng FastAPI để tạo API endpoint
   - `v3-streamlit-supabase-mem0.py` là ứng dụng web UI với Streamlit

2. **Quản lý người dùng**:
   - `studio-integration-version` sử dụng `session_id` và `user_id` để phân biệt người dùng
   - `v3-streamlit-supabase-mem0.py` quản lý người dùng qua UI đăng nhập nhưng không tách riêng hoàn toàn

3. **Lưu trữ tin nhắn và nhật ký**:
   - `studio-integration-version` lưu tin nhắn trong bảng "messages" trong Supabase
   - `v3-streamlit-supabase-mem0.py` chỉ lưu trong session state của Streamlit

## Cải tiến cần thực hiện cho v3-streamlit-supabase-mem0.py:



# 2. Nguyên lý của mem0 nằm ở hàm Memory.add() và Memory.search()

<think>
Let me analyze the `v3-streamlit-supabase-mem0.py` code to identify the functions responsible for extracting information and storing it in the database versus retrieving information from the database.

### Function for storing information in the DB:
In the `chat_with_memories` function, this part is responsible for storing new memories:

```python
# Create new memories from the conversation
try:
    messages.append({"role": "assistant", "content": assistant_response})
    memory.add(messages, user_id=user_id)
except Exception as e:
    st.warning(f"Could not save this conversation to memory: {str(e)}")
```

The key function here is `memory.add(messages, user_id=user_id)` which takes the entire conversation (including the new assistant response) and stores it in the database with the user's ID.

### Function for retrieving information from the DB:
Also in the `chat_with_memories` function, this part retrieves memories:

```python
# Retrieve relevant memories
with st.spinner("Searching memories..."):
    try:
        relevant_memories = memory.search(query=message, user_id=user_id, limit=3)
        memories_str = "\n".join(f"- {entry['memory']}" for entry in relevant_memories["results"])
    except Exception as e:
        st.error(f"Error retrieving memories: {str(e)}")
        memories_str = "(No memories available)"
```

The key function here is `memory.search(query=message, user_id=user_id, limit=3)` which searches for memories relevant to the current message for the given user.

Now, to update the `v3_view_database.py` file to show real-time updates of what's being stored in the database, I need to create a function that regularly polls the database for new entries or changes.
</think>

# Memory Storage and Retrieval Functions

## 1. Information Storage Function:
In the main application, the function that extracts and stores information in the database is:

```python
# Inside chat_with_memories function
memory.add(messages, user_id=user_id)
```

This function takes the entire conversation and saves it to the database. The actual implementation is inside the mem0 library, but it:
- Creates vector embeddings of the conversation
- Stores them in the "memories_new" collection
- Tags them with the user_id

## 2. Information Retrieval Function:
The function that retrieves information from the database is:

```python
# Inside chat_with_memories function
relevant_memories = memory.search(query=message, user_id=user_id, limit=3)
```

This searches the database for relevant memories and returns them.

## 3. Updated Database Viewer with Real-Time Updates

Let's update the database viewer to show real-time updates:

```python:mem0-agent/iterations/v3_view_database.py
import os
import streamlit as st
from dotenv import load_dotenv
import supabase
import pandas as pd
import time

# Load environment variables
load_dotenv()

# Initialize constants
SUPABASE_URL = os.environ.get("SUPABASE_URL", "")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY", "")
DB_URL = os.environ.get("DATABASE_URL", "")

# Streamlit page configuration
st.set_page_config(
    page_title="Memory Database Viewer",
    page_icon="🔍",
    layout="wide"
)

st.title("Memory Database Real-Time Viewer")

# Connect to vecs (the PostgreSQL extension used by mem0)
try:
    # Initialize Supabase client
    supabase_client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)
    st.success("Connected to Supabase!")
    
    # Show connection information
    st.write(f"Connection details:")
    st.write(f"- Supabase URL: {SUPABASE_URL[:20]}...")
    st.write(f"- Database URL exists: {'Yes' if DB_URL else 'No'}")
except Exception as e:
    st.error(f"Failed to connect to database: {str(e)}")
    st.stop()

# Function to get memories data
def get_memories_data():
    try:
        # Try different ways to access the data
        try:
            # Option 1: Direct SQL query using Supabase functions
            query = """
            SELECT id, metadata, created_at 
            FROM memories_new 
            ORDER BY created_at DESC 
            LIMIT 100
            """
            response = supabase_client.rpc('execute_sql', {'query': query}).execute()
            if response and response.data:
                return pd.DataFrame(response.data)
        except Exception as e1:
            st.error(f"Method 1 failed: {str(e1)}")
            
            try:
                # Option 2: Use Supabase table API
                response = supabase_client.table('memories_new').select('id,metadata,created_at').order('created_at', desc=True).limit(100).execute()
                if response and response.data:
                    return pd.DataFrame(response.data)
            except Exception as e2:
                st.error(f"Method 2 failed: {str(e2)}")
        
        # Both methods failed
        st.error("Could not access memories_new via standard methods")
        return pd.DataFrame()
    except Exception as e:
        st.error(f"General error fetching data: {str(e)}")
        return pd.DataFrame()

# Add automatic refresh option
auto_refresh = st.checkbox("Auto-refresh (every 5 seconds)", value=False)
refresh_button = st.button("Refresh Now")

# Place for memory count
memory_count = st.empty()

# Main data display area
data_display = st.container()

# Function to format and display memory data
def format_memory_data(df):
    if df.empty:
        return pd.DataFrame()
    
    # Extract useful information from metadata
    formatted_data = []
    for _, row in df.iterrows():
        item = {
            "id": row.get("id", ""),
            "timestamp": row.get("created_at", "")
        }
        
        # Extract from metadata if it exists and is a dict
        metadata = row.get("metadata", {})
        if isinstance(metadata, dict):
            item["user_id"] = metadata.get("user_id", "")
            item["memory"] = metadata.get("memory", "")
            # Add other metadata fields you want to display
        
        formatted_data.append(item)
    
    return pd.DataFrame(formatted_data)

# Main display loop
while True:
    # Get data
    if refresh_button or auto_refresh:
        with st.spinner("Loading latest memory data..."):
            df = get_memories_data()
        
        # Update the memory count
        memory_count.write(f"Total memories found: {len(df)}")
        
        # Display the data
        with data_display:
            if not df.empty:
                # Format the data for better display
                formatted_df = format_memory_data(df)
                if not formatted_df.empty:
                    st.subheader("Recent Memories")
                    st.dataframe(formatted_df, use_container_width=True)
                
                # Show the raw data in an expander
                with st.expander("View Raw Data"):
                    st.dataframe(df, use_container_width=True)
            else:
                st.warning("No memory data found. The database might be empty or inaccessible.")
    
    # Reset the refresh button
    refresh_button = False
    
    # If auto-refresh is enabled, wait 5 seconds before updating again
    if auto_refresh:
        time.sleep(5)
        st.rerun()
    else:
        break  # Exit the loop if not auto-refreshing
```


This updated viewer:
1. Shows memory data in real-time with an auto-refresh option
2. Extracts and formats the most important fields for easier reading
3. Provides both a clean formatted view and the raw data
4. Shows the total number of memories found
5. Offers multiple fallback methods to access the data

This will let you observe changes in real-time as your chat application adds new memories to the database.
